<a href="https://colab.research.google.com/github/Nageswar-Sahoo/Computer-Vision-Project/blob/main/S10/Untitled4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!git clone https://github.com/Nageswar-Sahoo/Computer-Vision-Project.git

Cloning into 'Computer-Vision-Project'...
remote: Enumerating objects: 1222, done.
remote: Counting objects: 100% (1222/1222), done.
remote: Compressing objects: 100% (986/986), done.
remote: Total 1222 (delta 772), reused 491 (delta 210), pack-reused 0
Receiving objects: 100% (1222/1222), 7.02 MiB | 11.58 MiB/s, done.
Resolving deltas: 100% (772/772), done.


In [6]:
!pip install -U git+https://github.com/albumentations-team/albumentations
import albumentations 
from albumentations.pytorch import ToTensorV2

  Cloning https://github.com/albumentations-team/albumentations to /tmp/pip-req-build-0702q6np
  Running command git clone -q https://github.com/albumentations-team/albumentations /tmp/pip-req-build-0702q6np


In [7]:
cd Computer-Vision-Project/template

/content/Computer-Vision-Project/template


In [8]:
! git clone https://github.com/seshuad/IMagenet
! ls 'IMagenet/tiny-imagenet-200/'

Cloning into 'IMagenet'...
remote: Enumerating objects: 120594, done.
remote: Total 120594 (delta 0), reused 0 (delta 0), pack-reused 120594
Receiving objects: 100% (120594/120594), 212.68 MiB | 17.29 MiB/s, done.
Resolving deltas: 100% (1115/1115), done.
Checking out files: 100% (120206/120206), done.
test  train  val  wnids.txt  words.txt


In [1]:
cd ..

/


In [2]:
cd /content

/content


In [3]:
!rm -rf Computer-Vision-Project

In [4]:
ls

IMagenet/  sample_data/


In [9]:
from parse_config import ConfigParser
from trainer import Trainer
from utils import prepare_device
from utils import get_splited_data
import model.loss as module_loss
import model.metric as module_metric
import torch.optim as optim
import logging
import numpy as np
import torch
import model.customresnet as module_resnet
import utils
SEED = 123
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(SEED)
import torch
import data_loader.data_loaders as data_loaders
import torchvision.models as models
import torch.nn as nn

In [10]:
    logger = logging.getLogger("trian")
    # Read the config.json
    config = ConfigParser.from_args()
    train_data_u, train_labels_u, test_data, test_labels = get_splited_data()
    train_labels_u=train_labels_u.astype(float)
    test_labels=test_labels.astype(float)




    data_loader = data_loaders.get_train_data_loader(train_data_u, train_labels_u, 64)

    valid_data_loader = data_loaders.get_test_data_loader(test_data, test_labels, 64)

    # build model architecture, then print to console
    model = module_resnet.CustomResnet()
    #Load Resnet18
    model = models.resnet18(True)
    #Finetune Final few layers to adjust for tiny imagenet input
    model.avgpool = nn.AdaptiveAvgPool2d(1)
    model.fc.out_features = 200



    logger.info(model)

    # prepare for (multi-device) GPU training
    n_gpu = 1
    device, device_ids = prepare_device(n_gpu)
    model = model.to(device)
    if len(device_ids) > 1:
        model = torch.nn.DataParallel(model, device_ids=device_ids)

    # get function handles of loss and metrics
    criterion = module_loss.crossentropyloss
    metrics = [module_metric.accuracy]

    optimizer = optim.Adam(model.parameters(), lr=.01, weight_decay=1e-4)
    # lr_scheduler = StepLR(optimizer, step_size=15, gamma=0.001)
    # optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
    scheduler = optim.lr_scheduler.OneCycleLR(optimizer, cycle_momentum=True, anneal_strategy='linear', verbose=False,
                                              three_phase=True, max_lr=.01, pct_start=5 / 24, epochs=24,
                                              steps_per_epoch=len(data_loader))

    trainer = Trainer(model, criterion, metrics, optimizer,
                      config=config,
                      device=device,
                      data_loader=data_loader,
                      valid_data_loader=valid_data_loader,
                      lr_scheduler=scheduler)



starting loading data
finished loading data, in 15.683183908462524 seconds
test data shape:  (30000, 64, 64, 3)
test_labels.shape:  (30000, 200)
train data shape:  (80000, 64, 64, 3)
train label shape:  (80000, 200)
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
     

In [ ]:
trainer.train()

Train Epoch: 1 last_lr_used  0.000402  [0/1250 (0%)] Loss: 15.254631
Train Epoch: 1 last_lr_used  0.000414  [8/1250 (1%)] Loss: 9.724217
Train Epoch: 1 last_lr_used  0.000426  [16/1250 (1%)] Loss: 6.633124
Train Epoch: 1 last_lr_used  0.000438  [24/1250 (2%)] Loss: 6.287477
Train Epoch: 1 last_lr_used  0.000451  [32/1250 (3%)] Loss: 5.825970
Train Epoch: 1 last_lr_used  0.000463  [40/1250 (3%)] Loss: 5.600729
Train Epoch: 1 last_lr_used  0.000475  [48/1250 (4%)] Loss: 5.567503
Train Epoch: 1 last_lr_used  0.000488  [56/1250 (4%)] Loss: 5.501855
Train Epoch: 1 last_lr_used  0.000500  [64/1250 (5%)] Loss: 5.316102
Train Epoch: 1 last_lr_used  0.000512  [72/1250 (6%)] Loss: 5.402303
Train Epoch: 1 last_lr_used  0.000524  [80/1250 (6%)] Loss: 5.539340
Train Epoch: 1 last_lr_used  0.000537  [88/1250 (7%)] Loss: 5.175946
Train Epoch: 1 last_lr_used  0.000549  [96/1250 (8%)] Loss: 5.266450
Train Epoch: 1 last_lr_used  0.000561  [104/1250 (8%)] Loss: 5.298096
Train Epoch: 1 last_lr_used  0.000